Select parameters based on Trackbars
- step1 separately
- step2 combined together

Order: Smoothing -> Histogramm Enhancement -> Sharpenging    

Chosen methods: guided filter -> CLAHE -> Homomorphic filter

In [67]:
# -*- encoding: utf-8 -*-
import sys
import cv2
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from typing import Tuple

# constant
TRACK_MAX_VAL = 100
# PATH
PATH_IMAGE = 'D:\\My_Data\\me_Docs\\Masterarbeit\\master_border_extraction\\images\\test1.png'
PATH_SMOOTH_OUTPUT = 'D:\\My_Data\\me_Docs\\Masterarbeit\\master_border_extraction\\data\\output\\preprocess_smooth\\'

# env: windows
img_bgr_origin = cv2.imread(PATH_IMAGE, cv2.IMREAD_COLOR)  # color image
img_gray_origin = cv2.imread(PATH_IMAGE, cv2.IMREAD_GRAYSCALE) # gray image
img_rgb = cv2.cvtColor(img_bgr_origin, cv2.COLOR_BGR2RGB)

In [4]:
def set_odd_val(num):
    if num % 2 == 0:
        num += 1
    return num

def set_two_decimal(num):
    return float(num)/100

In [2]:
print(img_gray_origin.shape)

(2064, 3088)


## 1 Smoothing

In [41]:
### test for `*` and `zip()`
def test(*params):
    print(params)
    res = zip(*params)
    print(list(res))

a = [1, 2]
b = [2, 3]
c = [3, 4]
test(a, b, c)

([1, 2], [2, 3], [3, 4])
[(1, 2, 3), (2, 3, 4)]


In [69]:
# meshgrid: N-D => extend to 1-D => combined to tuple  
def combine_grid_params(*param_lists) -> Tuple[list[tuple], int]:
    num_params = len(param_lists)

    param_grids = np.meshgrid(*param_lists, indexing='ij')
    param_flats = map(np.ndarray.flatten, param_grids)
    param_combine = list(zip(*param_flats))
    return param_combine, num_params

# example:
x = [1, 2, 3]
y = [4, 5, 6, 7]
res, cnt = combine_grid_params(x, y)
print(res, cnt)

[(1, 4), (1, 5), (1, 6), (1, 7), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7)] 2


Output for different sets of params

1 bilat filter

In [70]:
bilat_d,bilat_sigmaColor,bilat_sigmaSpace  = 33, 0.2, 30
img_gray_bilat = cv2.bilateralFilter(img_gray_origin, d=bilat_d, sigmaColor=bilat_sigmaColor, sigmaSpace=bilat_sigmaSpace)
cv2.imwrite(PATH_SMOOTH_OUTPUT+f'd={bilat_d}, sigmaColor={bilat_sigmaColor}, sigmaSpace={bilat_sigmaSpace}.png', img_gray_bilat)

True

In [71]:
### bilateralFilter()
img_gray_bilat = cv2.bilateralFilter(img_gray_origin, d=5, sigmaColor=20, sigmaSpace=20)

### params gridsearch => output: `PATH_SMOOTH_OUTPUT`
bilat_d = np.array([3, 5, 19, 33, 47], dtype=np.uint8)
bilat_sigmaColor = np.linspace(0.1, 0.5, 5)
bilat_sigmaSpace = np.linspace(10, 50, 5)

params, cnt = combine_grid_params(bilat_d, bilat_sigmaColor, bilat_sigmaSpace)
# check params number:
if cnt != 3:
    raise ValueError

for param in params:
    img_gray_bilat = cv2.bilateralFilter(img_gray_origin, d=param[0], sigmaColor=param[1], sigmaSpace=param[2])
    cv2.imwrite(PATH_SMOOTH_OUTPUT+f'bilat-d={param[0]} sigmaColor={param[1]} sigmaSpace={param[2]}.png', img_gray_bilat)

2 joinBilat filter

In [5]:
### jointBilateralFilter()
# gauss filter is used as guided image(joint)
img_gauss = cv2.GaussianBlur(img_gray_origin, ksize=(5, 5), sigmaX=1)
img_gray_jointBilat = cv2.ximgproc.jointBilateralFilter(img_gauss, img_gray_origin, d=5, sigmaColor=10, sigmaSpace=5)



3 guided filter

In [6]:
### GuidedFilter.filter()
img_gray_guided = cv2.ximgproc.guidedFilter(guide=img_rgb, 
                                            src=img_gray_origin,
                                            radius=10,
                                            eps=2)

In [7]:
print(img_gray_bilat.shape)
print(img_gray_jointBilat.shape)
print(img_gray_guided.shape)

(2064, 3088)
(2064, 3088)
(2064, 3088)


Function: trackbar()

1 trackbar() for bilateral filter

In [8]:
def update_all(trackPos):
    pass

cv2.namedWindow('Controls of Smoothing', cv2.WINDOW_NORMAL)
cv2.createTrackbar('bilat_d','Controls of Smoothing', 33, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('bilat_sigmaColor','Controls of Smoothing', 20, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('bilat_sigmaSpace','Controls of Smoothing', 30, TRACK_MAX_VAL, update_all)

figure_num = 0
while True:
    bilat_d = int(cv2.getTrackbarPos('bilat_d','Controls of Smoothing'))
    bilat_sigmaColor = float(cv2.getTrackbarPos('bilat_sigmaColor','Controls of Smoothing'))
    bilat_sigmaSpace = float(cv2.getTrackbarPos('bilat_sigmaSpace','Controls of Smoothing'))
    # adjust the params
    bilat_d = set_odd_val(bilat_d)
    bilat_sigmaColor = set_two_decimal(bilat_sigmaColor)

    # waitfor the user to press keys 
    k = cv2.waitKey(0) & 0xFF
    if k == 32:  # `Space` key
        ### bilateralFilter()
        img_gray_bilat = cv2.bilateralFilter(img_gray_origin, d=bilat_d, sigmaColor=bilat_sigmaColor, sigmaSpace=bilat_sigmaSpace)

        combined = np.hstack((img_gray_origin, img_gray_bilat))
        cv2.namedWindow(f'Smoothing {figure_num + 1}: d={bilat_d}, sigmaColor={bilat_sigmaColor}, sigmaSpace={bilat_sigmaSpace}', cv2.WINDOW_NORMAL)
        cv2.imshow(f'Smoothing {figure_num + 1}: d={bilat_d}, sigmaColor={bilat_sigmaColor}, sigmaSpace={bilat_sigmaSpace}', combined)
        figure_num +=1
    elif k == 27:  # `Esc` key
        break
cv2.destroyAllWindows()


2 trackbar() for joint bilateral filter

In [10]:
# trackbar callback fucntion does nothing
def update_all(x):
    pass

cv2.namedWindow('Controls of Smoothing', cv2.WINDOW_NORMAL)

cv2.createTrackbar('gauss_ksize','Controls of Smoothing', 33, TRACK_MAX_VAL, update_all)   # !!!parameter restriction: positive & odd
cv2.createTrackbar('gauss_sigma','Controls of Smoothing', 7, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('jointBilat_d','Controls of Smoothing', 33, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('jointBilat_sigmaColor','Controls of Smoothing', 100, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('jointBilat_sigmaSpace','Controls of Smoothing', 7, TRACK_MAX_VAL, update_all)

figure_num = 0
while True:
    gauss_ksize = int(cv2.getTrackbarPos('gauss_ksize','Controls of Smoothing'))
    gauss_sigma = int(cv2.getTrackbarPos('gauss_sigma','Controls of Smoothing'))
    jointBilat_d = int(cv2.getTrackbarPos('jointBilat_d','Controls of Smoothing'))
    jointBilat_sigmaColor = int(cv2.getTrackbarPos('jointBilat_sigmaColor','Controls of Smoothing'))
    jointBilat_sigmaSpace = int(cv2.getTrackbarPos('jointBilat_sigmaSpace','Controls of Smoothing'))
    # adjust the params
    gauss_ksize = set_odd_val(gauss_ksize)
    jointBilat_d = set_odd_val(jointBilat_d)
    jointBilat_sigmaColor = set_two_decimal(jointBilat_sigmaColor)
    
    # waitfor the user to press keys 
    k = cv2.waitKey(0) & 0xFF
    if k == 32:  # `Space` key
        ### jointBilateralFilter()
        # gauss filter is used as guided image(joint)
        img_gauss = cv2.GaussianBlur(img_gray_origin, ksize=(gauss_ksize, gauss_ksize), sigmaX=gauss_sigma)
        img_gray_jointBilat = cv2.ximgproc.jointBilateralFilter(img_gauss, img_gray_origin, d=jointBilat_d, sigmaColor=jointBilat_sigmaColor, sigmaSpace=jointBilat_sigmaSpace)
        
        combined = np.hstack((img_gray_origin, img_gray_jointBilat))
        cv2.namedWindow(f'Smoothing {figure_num + 1}, GaussSize={gauss_ksize, gauss_ksize}, GaussSigma={gauss_sigma}, d={jointBilat_d}, sigmaColor={jointBilat_sigmaColor}, sigmaSpace={jointBilat_sigmaSpace}', cv2.WINDOW_NORMAL)
        cv2.imshow(f'Smoothing {figure_num + 1}, GaussSize={gauss_ksize, gauss_ksize}, GaussSigma={gauss_sigma}, d={jointBilat_d}, sigmaColor={jointBilat_sigmaColor}, sigmaSpace={jointBilat_sigmaSpace}', combined)
        figure_num +=1
    elif k == 27:  # `Esc` key
        break
cv2.destroyAllWindows()


3 trackbar() for guided filter

In [11]:
# trackbar callback fucntion does nothing
def update_all(x):
    pass

cv2.namedWindow('Controls of Smoothing', cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Controls of Smoothing", 300, 300)  

cv2.createTrackbar('guide_radius','Controls of Smoothing', 14, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('guide_eps','Controls of Smoothing', 20, TRACK_MAX_VAL, update_all)   # square is like `sigmaColor`

figure_num = 0
while True:
    guide_radius = int(cv2.getTrackbarPos('guide_radius','Controls of Smoothing'))
    guide_eps = int(cv2.getTrackbarPos('guide_eps','Controls of Smoothing'))
    # adjust the params
    guide_eps = set_two_decimal(guide_eps)
    
    # waitfor the user to press keys 
    k = cv2.waitKey(0) & 0xFF
    if k == 32:  # `Space`
        ### GuidedFilter.filter()
        img_gray_guided = cv2.ximgproc.guidedFilter(guide=img_rgb, 
                                                src=img_gray_origin,
                                                radius=guide_radius,
                                                eps=guide_eps)
            
        combined = np.hstack((img_gray_origin, img_gray_guided))
        cv2.namedWindow(f'Smoothing {figure_num + 1}, radius={guide_radius}, eps={guide_eps}', cv2.WINDOW_NORMAL)
        cv2.imshow(f'Smoothing {figure_num + 1}, radius={guide_radius}, eps={guide_eps}', combined)
        figure_num +=1
    elif k == 27:  # `Esc`
        break
cv2.destroyAllWindows()


!!!combine all together: not so obvious => It's better see the effect separately

In [ ]:
def update_all(trackPos):
    bilat_d = int(cv2.getTrackbarPos('bilat_d','Controls of Smoothing'))
    bilat_sigmaColor = int(cv2.getTrackbarPos('bilat_sigmaColor','Controls of Smoothing'))
    bilat_sigmaSpace = int(cv2.getTrackbarPos('bilat_sigmaSpace','Controls of Smoothing'))
    ### bilateralFilter()
    img_gray_bilat = cv2.bilateralFilter(img_gray_origin, d=bilat_d, sigmaColor=bilat_sigmaColor, sigmaSpace=bilat_sigmaSpace)


    gauss_ksize = int(cv2.getTrackbarPos('gauss_ksize','Controls of Smoothing'))
    if gauss_ksize % 2 == 0:
        gauss_ksize = gauss_ksize + 1

    gauss_sigma = int(cv2.getTrackbarPos('gauss_sigma','Controls of Smoothing'))
    jointBilat_d = int(cv2.getTrackbarPos('jointBilat_d','Controls of Smoothing'))
    jointBilat_sigmaColor = int(cv2.getTrackbarPos('jointBilat_sigmaColor','Controls of Smoothing'))
    jointBilat_sigmaSpace = int(cv2.getTrackbarPos('jointBilat_sigmaSpace','Controls of Smoothing'))
    ### jointBilateralFilter()
    # gauss filter is used as guided image(joint)
    img_gauss = cv2.GaussianBlur(img_gray_origin, ksize=(gauss_ksize, gauss_ksize), sigmaX=gauss_sigma)
    img_gray_jointBilat = cv2.ximgproc.jointBilateralFilter(img_gauss, img_gray_origin, d=jointBilat_d, sigmaColor=jointBilat_sigmaColor, sigmaSpace=jointBilat_sigmaSpace)


    guide_radius = int(cv2.getTrackbarPos('guide_radius','Controls of Smoothing'))
    guide_eps = int(cv2.getTrackbarPos('guide_eps','Controls of Smoothing'))
    ### GuidedFilter.filter()
    img_gray_guided = cv2.ximgproc.guidedFilter(guide=img_rgb, 
                                            src=img_gray_origin,
                                            radius=guide_radius,
                                            eps=guide_eps)

    img_combined = np.hstack((img_gray_origin, img_gray_bilat, img_gray_jointBilat, img_gray_guided))

    cv2.imshow('Controls of Smoothing', img_combined)


cv2.namedWindow('Controls of Smoothing', cv2.WINDOW_NORMAL)

cv2.createTrackbar('bilat_d','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('bilat_sigmaColor','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('bilat_sigmaSpace','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)

cv2.createTrackbar('gauss_ksize','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)   # !!!parameter restriction: positive & odd
cv2.createTrackbar('gauss_sigma','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('jointBilat_d','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('jointBilat_sigmaColor','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('jointBilat_sigmaSpace','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)

cv2.createTrackbar('guide_radius','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)
cv2.createTrackbar('guide_eps','Controls of Smoothing', 0, TRACK_MAX_VAL, update_all)   # square is like `sigmaColor`

while True:
    # waitfor the user to press keys 
    k = cv2.waitKey(0) & 0xFF
    if k == 32:  # `Space`
        ### bilateralFilter()
        img_gray_bilat = cv2.bilateralFilter(img_gray_origin, d=bilat_d, sigmaColor=bilat_sigmaColor, sigmaSpace=bilat_sigmaSpace)

        img_combined = np.hstack((img_gray_origin, img_gray_bilat))
        cv2.imshow('Controls of Smoothing', img_combined)
    if k == 27:  # `Esc`
        print(-1, bilat_d, bilat_sigmaColor, bilat_sigmaSpace)
        break
cv2.destroyAllWindows()